# **Assignment 2**

## Yingjie Chen (yc24966) - 08:30 section
## Andrew Han (ah49372) - 10:30 section
## Sitong Li (sl43736) - 10:30 section
## Matthew Ruffner (mar9632) - 08:30 section
## Xinmeng Song (xs3659) - 10:30 section

###Preparation

In [ ]:
!pip install selenium
from selenium import webdriver
!apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip3 install beautifulsoup4 requests

     |████████████████████████████████| 911kB 2.8MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.u

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup


###Task A: Scraping Beer Data

In [ ]:

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()

In [ ]:
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
# d = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver',chrome_options=chrome_options)
# d.get('https://www.google.nl/')

In [ ]:
wd = webdriver.Chrome('chromedriver', options=chrome_options)


In [ ]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

url = "https://www.beeradvocate.com/beer/top-rated/" #main page
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')


In [ ]:
#Create a url_list

import re

url_list = []
count = 0
for link in soup.find_all('a'):
    pattern = re.compile("/beer/profile/\d+/\d+")

    if pattern.match(str(link.get('href'))):
      #print(str(link.get('href'))) 
      count += 1
      url_list.append("https://www.beeradvocate.com/"+ str(link.get('href')))

count

In [ ]:
#/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[7]/b
total_number_reviews = wd.find_elements_by_xpath("/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[7]/b")[0]
total_number_reviews.text
total_number = int(re.findall("\d+", total_number_reviews.text)[0])

21

In [ ]:
x

26

In [ ]:
total_comments = pd.DataFrame(columns = ['product_name','product_review','user_rating'])
for url in url_list:

    web_address = url #"https://www.beeradvocate.com/beer/profile/23222/78820/"
    wd.get(web_address)
    #print(web_address)

    comments = pd.DataFrame(columns = ['product_name','product_review','user_rating'])
    ids = wd.find_elements_by_xpath("/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[1]")  #//*[@id="rating_fullview_container"] /html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div[1]
    comment_ids = []

    for i in ids:
        comment_ids.append(i.get_attribute('id'))
    comment_ids

    #put these examples into a for loop for kentucky beer, 0-24 (25 total comments)
    product_name = wd.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[1]/h1')[0]
    name = product_name.text
    #print(name)

    total_number_reviews = wd.find_elements_by_xpath("/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[7]/b")[0]
    total_number_reviews.text
    total_number = int(re.findall("\d+", total_number_reviews.text)[0])
    if total_number > 25:
        total_number = 25
    for x in range(1,total_number+1):
        user_rating = wd.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div['+ str(x)+']/div[2]/span[2]')[0]
        rating = user_rating.text
        #rating

        product_review = wd.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div['+ str(x)+']/div[2]')[0] 
        review = product_review.text #how to deal with text() object at the end instead of an element???
        #review

        #add these to a dataframe
        comments.loc[len(comments)] = [name,review,rating]

    total_comments = total_comments.append(comments, ignore_index=True)
    

In [ ]:
#print(total_comments.tail())
type(total_comments)
total_comments.groupby("product_name").count()
len(total_comments)

5743

In [ ]:
total_comments.to_csv("HW2_results_1.csv")
!ls

HW2_results_1.csv  sample_data


###Task B: Cleaning and Word Frequency:

In [ ]:
#import csv files from above and car-brand one from Canvas
from google.colab import files
uploaded = files.upload()

Saving HW2_results_1.csv to HW2_results_1.csv


In [ ]:
reviews = pd.read_csv("HW2_results_1.csv").drop(['Unnamed: 0'],axis=1)

In [ ]:
reviews.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout\nToppling Goliath ...,HOPTOMIC_BOMB from California\n\n5/5 rDev +2%...,5.00
1,Kentucky Brunch Brand Stout\nToppling Goliath ...,Prospero from Colorado\n\n4.53/5 rDev -7.6%\n...,4.53
2,Kentucky Brunch Brand Stout\nToppling Goliath ...,BourbonForBeer from Illinois\n\n1.49/5 rDev -...,1.49
3,Kentucky Brunch Brand Stout\nToppling Goliath ...,JellybonesElmer from Wisconsin\n\n4.52/5 rDev...,4.52
4,Kentucky Brunch Brand Stout\nToppling Goliath ...,WormGod from New York\n\n4.99/5 rDev +1.8%\nl...,4.99


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

def lower(string):
    return str(string).lower()
reviews["product_review"] = reviews["product_review"].apply(lower)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def findre(string):
    return re.findall("([a-zA-Z]+)", string)
reviews["list_of_words"] = reviews["product_review"].apply(findre)
reviews["list_of_words"]

stop_words = set(stopwords.words('english'))

def remove_single(s):
    return [w for w in s if not w in list(map(chr, range(97, 123)))] 

def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 

reviews["final_version"] = reviews["list_of_words"].apply(remove_stopwords)
reviews["final_version"] = reviews["final_version"].apply(remove_single)

In [ ]:
def review_list(lst):
    if "overall" in lst:
        ind = lst.index("overall")
        return lst[ind+1:]
    return lst

reviews["final_version"] = reviews["final_version"].map(review_list)
reviews["final_version"]

0       [smell, early, morning, pancakes, coffee, work...
1       [vintage, pours, dark, brown, color, noticeabl...
2       [hyped, lot, breweries, style, beer, right, to...
3       [reviewing, vintage, pours, thick, close, blac...
4       [version, poured, dark, small, head, smell, ma...
                              ...                        
5738    [pours, dark, chestnut, brown, two, half, fing...
5739    [vintage, heavy, booze, aroma, vanilla, accent...
5740    [opaque, near, black, body, frothy, cola, like...
5741    [drank, pint, fl, oz, bottle, purchased, craft...
5742    [pours, black, eggshell, thin, layer, khaki, f...
Name: final_version, Length: 5743, dtype: object

In [ ]:
def word_count(words_list):
    dictionary = {}
    for words in words_list:
        if words not in dictionary.keys():
            dictionary[words] = 1
    return dictionary

reviews["dictionary_version"] = reviews["final_version"].apply(word_count)

In [ ]:
reviews["dictionary_version"]

0       {'smell': 1, 'early': 1, 'morning': 1, 'pancak...
1       {'vintage': 1, 'pours': 1, 'dark': 1, 'brown':...
2       {'hyped': 1, 'lot': 1, 'breweries': 1, 'style'...
3       {'reviewing': 1, 'vintage': 1, 'pours': 1, 'th...
4       {'version': 1, 'poured': 1, 'dark': 1, 'small'...
                              ...                        
5738    {'pours': 1, 'dark': 1, 'chestnut': 1, 'brown'...
5739    {'vintage': 1, 'heavy': 1, 'booze': 1, 'aroma'...
5740    {'opaque': 1, 'near': 1, 'black': 1, 'body': 1...
5741    {'drank': 1, 'pint': 1, 'fl': 1, 'oz': 1, 'bot...
5742    {'pours': 1, 'black': 1, 'eggshell': 1, 'thin'...
Name: dictionary_version, Length: 5743, dtype: object

In [ ]:
final_dict = {}

def update_dict(dictionary):
    for key in dictionary.keys():
        if key in final_dict.keys():
            final_dict[key] += 1
        else:
            final_dict[key] = 1
        
reviews["dictionary_version"].apply(update_dict)

0       None
1       None
2       None
3       None
4       None
        ... 
5738    None
5739    None
5740    None
5741    None
5742    None
Name: dictionary_version, Length: 5743, dtype: object

In [ ]:
df = pd.DataFrame.from_dict(final_dict, orient='index')
df.head()

,0
smell,987
early,35
morning,35
pancakes,18
coffee,959


In [ ]:
# df.rename(columns = "0", inplace = True)
word_freq_list = df[0].sort_values(ascending=False).index.tolist()
word_freq_list[0:20]

['head',
 'beer',
 'taste',
 'nose',
 'sweet',
 'like',
 'pours',
 'dark',
 'one',
 'finish',
 'carbonation',
 'chocolate',
 'medium',
 'aroma',
 'well',
 'mouthfeel',
 'overall',
 'nice',
 'body',
 'good']

In [ ]:
beer_attri = ["aggressive", "balanced", "complex", "crisp", "fruity", "hoppy", "malty", "robust"]

In [ ]:
top_brands = []
for name in word_freq_list:
    if name in beer_attri:
        top_brands.append(name)
top_list = top_brands[0:8]
top_list

['balanced',
 'complex',
 'fruity',
 'crisp',
 'hoppy',
 'malty',
 'robust',
 'aggressive']

In [ ]:
def findcount(string):
    return df.loc[string]
top_list = pd.DataFrame(top_list)
top_list["count"] = top_list[0].apply(findcount)
top_list[0:3]
#These are the top 3 mentioned beer attributes.
#We gonna assume the user select these 3 attributes

,0,count
0,balanced,693
1,complex,477
2,fruity,431


###Task C: Cosine Similarity:
Questions:
1. CountVectorizer VS TfidfVectorizer
2. Can we pass the list of words into comparison?
3. also, what does it really mean the similarity score between a "3-word-sentence" and a vectors of say 100 words?
4. Also, output should be the average score for each beer, not each review.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tmp = ["balanced", "complex", "fruity"]
attribute = ", ".join(tmp)

# df = pd.DataFrame({"review": ["I love the taste, and color, smell of this beer"]})
# text2 = "I love the taste, and color, smell of this beer"

def similarity(list_of_words):
    string = ", ".join(list_of_words)
    documents = [attribute, string]
    count_vectorizer = CountVectorizer()
    count_matrix = count_vectorizer.fit_transform(documents)
    #print(count_matrix)
    return cosine_similarity(count_matrix[0:1], count_matrix)[0][1]

reviews["similarity"] = reviews["final_version"].map(similarity)

In [ ]:
#checking
reviews["similarity"].sort_values(ascending=False)

552     0.316228
3674    0.316228
3420    0.298142
2035    0.288675
5352    0.280056
          ...   
3566    0.000000
3567    0.000000
3568    0.000000
3570    0.000000
0       0.000000
Name: similarity, Length: 5743, dtype: float64

In [ ]:
# reviews1 = reviews.drop(['product_review','user_rating','list_of_words','dictionary_version'], axis = 1)
# reviews1
reviews[["product_name","final_version", "similarity"]]

,product_name,final_version,similarity
0,Kentucky Brunch Brand Stout\nToppling Goliath ...,"[smell, early, morning, pancakes, coffee, work...",0.000000
1,Kentucky Brunch Brand Stout\nToppling Goliath ...,"[vintage, pours, dark, brown, color, noticeabl...",0.000000
2,Kentucky Brunch Brand Stout\nToppling Goliath ...,"[hyped, lot, breweries, style, beer, right, to...",0.083333
3,Kentucky Brunch Brand Stout\nToppling Goliath ...,"[reviewing, vintage, pours, thick, close, blac...",0.000000
4,Kentucky Brunch Brand Stout\nToppling Goliath ...,"[version, poured, dark, small, head, smell, ma...",0.000000
...,...,...,...
5738,Madagascar\n4 Hands Brewing Co.,"[pours, dark, chestnut, brown, two, half, fing...",0.000000
5739,Madagascar\n4 Hands Brewing Co.,"[vintage, heavy, booze, aroma, vanilla, accent...",0.000000
5740,Madagascar\n4 Hands Brewing Co.,"[opaque, near, black, body, frothy, cola, like...",0.000000
5741,Madagascar\n4 Hands Brewing Co.,"[drank, pint, fl, oz, bottle, purchased, craft...",0.025616


### Task D: Sentiment

In [ ]:
from collections import Counter
features = tmp
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
import numpy as np

def once(review, ori1, ori2):
#     print(ori1, ori2)
    startpos = ori1-2 if ori1-2 > 0 else 0
    endpos = ori2+2 if ori2+2 < len(review)-1 else len(review)-1
    sentence = review[startpos: endpos+1]
    sentence_s = " ".join(sentence)
#     print(sentence_s)
    return sid.polarity_scores(sentence_s)["compound"]

for number in range(3):

  def sentiment(review):
    if features[number] in review:
        scores = []
        count = [i for i, n in enumerate(review) if n == features[number]]
        return once(review, count[0], count[-1])
    return 0

  reviews[features[number]] = reviews['final_version'].map(sentiment)


reviews.replace(0, np.NaN)
#df[["score", "score-1","score-2"]].mean(axis=1)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


,product_name,product_review,user_rating,list_of_words,final_version,dictionary_version,similarity,balanced,complex,fruity
0,Kentucky Brunch Brand Stout\nToppling Goliath ...,hoptomic_bomb from california\n\n5/5 rdev +2%...,5.00,"[hoptomic, bomb, from, california, rdev, look,...","[smell, early, morning, pancakes, coffee, work...","{'smell': 1, 'early': 1, 'morning': 1, 'pancak...",NaN,NaN,NaN,NaN
1,Kentucky Brunch Brand Stout\nToppling Goliath ...,prospero from colorado\n\n4.53/5 rdev -7.6%\n...,4.53,"[prospero, from, colorado, rdev, look, smell, ...","[vintage, pours, dark, brown, color, noticeabl...","{'vintage': 1, 'pours': 1, 'dark': 1, 'brown':...",NaN,NaN,NaN,NaN
2,Kentucky Brunch Brand Stout\nToppling Goliath ...,bourbonforbeer from illinois\n\n1.49/5 rdev -...,1.49,"[bourbonforbeer, from, illinois, rdev, look, s...","[hyped, lot, breweries, style, beer, right, to...","{'hyped': 1, 'lot': 1, 'breweries': 1, 'style'...",0.083333,0.7184,NaN,NaN
3,Kentucky Brunch Brand Stout\nToppling Goliath ...,jellyboneselmer from wisconsin\n\n4.52/5 rdev...,4.52,"[jellyboneselmer, from, wisconsin, rdev, look,...","[reviewing, vintage, pours, thick, close, blac...","{'reviewing': 1, 'vintage': 1, 'pours': 1, 'th...",NaN,NaN,NaN,NaN
4,Kentucky Brunch Brand Stout\nToppling Goliath ...,wormgod from new york\n\n4.99/5 rdev +1.8%\nl...,4.99,"[wormgod, from, new, york, rdev, look, smell, ...","[version, poured, dark, small, head, smell, ma...","{'version': 1, 'poured': 1, 'dark': 1, 'small'...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5738,Madagascar\n4 Hands Brewing Co.,thebeers from pennsylvania\n\n4.07/5 rdev -8....,4.07,"[thebeers, from, pennsylvania, rdev, look, sme...","[pours, dark, chestnut, brown, two, half, fing...","{'pours': 1, 'dark': 1, 'chestnut': 1, 'brown'...",NaN,NaN,NaN,NaN
5739,Madagascar\n4 Hands Brewing Co.,furlinator from california\n\n4.15/5 rdev -7%...,4.15,"[furlinator, from, california, rdev, look, sme...","[vintage, heavy, booze, aroma, vanilla, accent...","{'vintage': 1, 'heavy': 1, 'booze': 1, 'aroma'...",NaN,NaN,NaN,NaN
5740,Madagascar\n4 Hands Brewing Co.,huhzubendah from district of columbia\n\n4.21/...,4.21,"[huhzubendah, from, district, of, columbia, rd...","[opaque, near, black, body, frothy, cola, like...","{'opaque': 1, 'near': 1, 'black': 1, 'body': 1...",NaN,NaN,NaN,NaN
5741,Madagascar\n4 Hands Brewing Co.,damian from massachusetts\n\n4.61/5 rdev +3.4...,4.61,"[damian, from, massachusetts, rdev, look, smel...","[drank, pint, fl, oz, bottle, purchased, craft...","{'drank': 1, 'pint': 1, 'fl': 1, 'oz': 1, 'bot...",0.025616,0.2732,NaN,NaN


In [ ]:
#checking
reviews["fruity"].sort_values()

960    -0.6931
5595   -0.6808
442    -0.4939
5543   -0.4588
3793   -0.4404
         ...  
1337    0.9879
1964    0.9882
3133    0.9896
2924    0.9902
3250    0.9931
Name: fruity, Length: 5743, dtype: float64

### Task E: Evaluation Scores



In [ ]:
reviews["sentiment_score"]=reviews.apply(lambda x:(x["balanced"]+x["complex"]+x["fruity"])/3,axis=1)

In [ ]:
reviews_group_df=reviews.loc[:,["product_name","user_rating","sentiment_score","similarity"]].groupby("product_name").mean().reset_index()

In [ ]:
reviews_group_df["evaluation_score"]=reviews_group_df.apply(lambda x:x["similarity"]+x["sentiment_score"],axis=1)
reviews_group_df.sort_values(by='evaluation_score',ascending=False)[0:3]

#Our Top 3 Recommendations:

,product_name,user_rating,sentiment_score,similarity,evaluation_score
66,Consecration\nRussian River Brewing Company,4.48,0.286367,0.142134,0.428500
225,Temptation\nRussian River Brewing Company,4.20,0.212300,0.067574,0.279874
40,Bomb!\nPrairie Artisan Ales,4.75,0.190633,0.072169,0.262802


### Task F: Comparing Bag of Words and Word Vectors


In [ ]:
# Code
!python3 -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.5MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=e684e62e2b11d32303d1c58e7d26908c4b484f58f3b338333c46d852f79e5abf
  Stored in directory: /tmp/pip-ephem-wheel-cache-n_n4wkky/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import en_core_web_md
import spacy
nlp = en_core_web_md.load()

In [ ]:
tmp = ["balance", "complex", "fruity"]
attributes = nlp(", ".join(tmp))

def word_embed_sim(doc):
    doc_str = nlp(", ".join(doc))
    return doc_str.similarity(attributes)

reviews['word embedding similarity'] = reviews["final_version"].map(word_embed_sim)

In [ ]:
reviews['word embedding similarity']

0       0.823762
1       0.844765
2       0.796737
3       0.821957
4       0.821838
          ...   
5738    0.853914
5739    0.849099
5740    0.838608
5741    0.846600
5742    0.847185
Name: word embedding similarity, Length: 5743, dtype: float64

In [ ]:
# Table 1: Top3 recommended beer using Bag of Word Cosine similarity
# Column0: top 3 product names
# Column1: average similarity score for product (#1,2,3)
# Column2: % reviews mentioning "balanced" 
# Column3: % reviews mentioning "complex"
# Column4: % reviews mentioning "fruity"

#This should be a 5 X 4 tables, including header and column names
top_sim = reviews_group_df.sort_values(by='similarity',ascending=False)[0:6][["product_name","user_rating","similarity"]].copy()
top_sim

,product_name,user_rating,similarity
66,Consecration\nRussian River Brewing Company,4.4800,0.142134
40,Bomb!\nPrairie Artisan Ales,4.7500,0.072169
185,Oude Geuze Golden Blend\nBrouwerij 3 Fonteinen,4.5632,0.069557
225,Temptation\nRussian River Brewing Company,4.2000,0.067574
125,Gueuze 100% Lambic\nBrasserie Cantillon,4.3912,0.064257
247,Zenne Y Frontera\nBrouwerij 3 Fonteinen,4.7068,0.063609


In [ ]:
def percent_in_reviews(attri, product):
    # product = "Oude Geuze Golden Blend\nBrouwerij 3 Fonteinen"
    # attri = "balanced"
    df_temp = reviews[reviews["product_name"] == product]
    df_temp

    count = 0
    for list_words in df_temp["final_version"]: 
        if attri in list_words:
            count += 1
    return (count/len(df_temp["final_version"]))

for attri in tmp:
    top_sim["% reviews mentioning " + attri] = [0,0,0,0,0,0]
  
count_row = 0
count_col = 0

for product in top_sim["product_name"]:
    count_col = 0
    for attri in tmp:
        top_sim.iloc[count_row, top_sim.columns.get_loc('% reviews mentioning ' + attri)]= percent_in_reviews(attri, product)
        count_col += 1
    count_row += 1

In [ ]:
top_sim

,product_name,user_rating,similarity,% reviews mentioning balance,% reviews mentioning complex,% reviews mentioning fruity
66,Consecration\nRussian River Brewing Company,4.4800,0.142134,1.00,1.00,0.00
40,Bomb!\nPrairie Artisan Ales,4.7500,0.072169,0.00,1.00,0.00
185,Oude Geuze Golden Blend\nBrouwerij 3 Fonteinen,4.5632,0.069557,0.04,0.28,0.20
225,Temptation\nRussian River Brewing Company,4.2000,0.067574,0.00,0.00,1.00
125,Gueuze 100% Lambic\nBrasserie Cantillon,4.3912,0.064257,0.16,0.28,0.24
247,Zenne Y Frontera\nBrouwerij 3 Fonteinen,4.7068,0.063609,0.00,0.28,0.16


In [ ]:
# Table 2: Top3 recommended beer using word vectors (spaCy)
# Column0: top 3 product names
# Column1: average similarity score for product 1
# Column2: % reviews mentioning "balanced" 
# Column3: % reviews mentioning "complex"
# Column4: % reviews mentioning "fruity"
reviews_group_df2 =reviews.loc[:,["product_name","user_rating","sentiment_score","word embedding similarity"]].groupby("product_name").mean().reset_index()
top_sim2 = reviews_group_df2.sort_values(by='word embedding similarity',ascending=False)[0:5][["product_name","user_rating","word embedding similarity"]].copy()
top_sim2

,product_name,user_rating,word embedding similarity
66,Consecration\nRussian River Brewing Company,4.4800,0.867107
249,§ucaba\nFirestone Walker Brewing Co.,4.5000,0.851568
222,Supplication\nRussian River Brewing Company,4.4000,0.849155
248,Zombie Dust\n3 Floyds Brewing Co.,4.4200,0.847071
45,Bourbon Barrel Oro Negro\nJackie O's Pub & Bre...,4.4984,0.847033


In [ ]:
for attri in tmp:
    top_sim2["% reviews mentioning " + attri] = [0,0,0,0,0]
  
count_row = 0
count_col = 0

for product in top_sim2["product_name"]:
    count_col = 0
    for attri in tmp:
        top_sim2.iloc[count_row, top_sim2.columns.get_loc('% reviews mentioning ' + attri)]= percent_in_reviews(attri, product)
        count_col += 1
    count_row += 1

top_sim2

,product_name,user_rating,word embedding similarity,% reviews mentioning balance,% reviews mentioning complex,% reviews mentioning fruity
66,Consecration\nRussian River Brewing Company,4.4800,0.867107,1.00,1.0,0.0
249,§ucaba\nFirestone Walker Brewing Co.,4.5000,0.851568,0.00,0.0,0.0
222,Supplication\nRussian River Brewing Company,4.4000,0.849155,0.00,0.0,1.0
248,Zombie Dust\n3 Floyds Brewing Co.,4.4200,0.847071,0.00,0.0,0.0
45,Bourbon Barrel Oro Negro\nJackie O's Pub & Bre...,4.4984,0.847033,0.28,0.2,0.0


**Compare the results and talk about why**  
For cosine similarity (plain vanilla bag-of-words), the list of top five brands we get is different from that with word vector similarity. The team used top five brands instead of top three for both tables as some brands did not have enough reviews to run the analysis which distorted the average similarity score. 

On the other hand, when the team ran spaCy and ordered the dataset by "word-embedding similarity" (word vectors), the result shows that most reviews does not mention the exact feature words. However, the team was able to observe that they have a higher similarity score because they mention something similar to the three features that was assigned.

### Task G: Comparing User Rating and Our Recommendations

In [ ]:
#Just by user rating
reviews_group_df.sort_values(by='user_rating',ascending=False)[0:5]

,product_name,user_rating,sentiment_score,similarity,evaluation_score
232,Trappist Westvleteren 12 (XII)\nBrouwerij West...,5.000000,0.000000,0.000000,0.000000
52,CBS (Canadian Breakfast Stout)\nFounders Brewi...,4.890000,0.000000,0.000000,0.000000
48,Bourbon County Brand Stout\nGoose Island Beer Co.,4.880000,0.000000,0.000000,0.000000
47,Bourbon County Brand Coffee Stout\nGoose Islan...,4.770000,0.000000,0.000000,0.000000
56,Chemtrailmix\n3 Floyds Brewing Co.,4.754286,0.035895,0.024433,0.060328


Discusssion here:
Select 3 highest rated products from the entire dataset are:

**1.Trappist Westvleteren 12 (XII) --Brouwerij Westvleteren (Sint-Sixtusabdij van Westvleteren)**

**2.CBS (Canadian Breakfast Stout) --Founders Brewing Company**

**3.Bourbon County Brand Stout --Goose Island Beer Co.**

The product with the user rating is Trappist Westvleteren 12 (XII). From the dataset, it can be seen that this product has only one piece of review, and the score is 5 points. It can be seen from the review of the product that the customer highly recommends this product, but the features "balanced", "complex" and "fruity" we selected is not mentioned in the product's review, so the evaluation_score is 0.

The second-ranked product is CBS (Canadian Breakfast Stout). This product has only one piece of data in the crawled data set. Customers think Not very good with smells but you can definitely smell maple in there. It has nothing to do with fruity characteristics. The other two features are also not mentioned explicitly.

The third product is Bourbon County Brand Stout, which has 3 reviews. Customers think that the taste is sublime and ever-changing. But it does not mention the features we selected.

###Conclusion:
If we recommend the customer who's using this system the beer with top "user_rating" without taking considerations of cosine similarity score/feature sentiment, we would fail to make the best suggestions based on his/her specific tastes (features). Also, the main reason that these "top beers" have such high user-rating is because there are not enough reviews that are written for some products (small sample data). Therefore, they ended up being ranked top in the chart but with a evaluation score of 0.

